In [3]:
import pandas as pd
import numpy as np
import os

# --- Configuración ---
# Define la ruta del archivo de entrada
input_file_path = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_restaurantes_con_ids.csv'

# Define la ruta del archivo de salida
input_dir, input_filename = os.path.split(input_file_path)
base_name, ext = os.path.splitext(input_filename)
output_filename = f"{base_name}_procesado{ext}" # Nuevo nombre para el archivo procesado
output_file_path = os.path.join(input_dir, output_filename)

# Lista de columnas que quieres conservar
columns_to_keep = [
    'id_nombre', 'nombre_restaurante', 'calificacion', 'puntuacion',
    'tipo_cocina', 'distrito', 'edificio', 'calle', 'codigo_postal',
    'telefono', 'latitud', 'longitud', 'punto_ubicacion',
    'distrito_comunitario', 'distrito_consejo', 'zona_censo',
    'fecha_inspeccion' # Aseguramos que esta columna esté incluida
]

# Columnas necesarias para crear la dirección completa
address_cols = ['edificio', 'calle', 'distrito', 'codigo_postal']
# --- Fin Configuración ---

try:
    # Leer el archivo CSV
    print(f"Leyendo archivo: {input_file_path}")
    df = pd.read_csv(input_file_path, low_memory=False)
    print("Archivo leído correctamente.")

    # Verificar si existen todas las columnas necesarias para la dirección
    if all(col in df.columns for col in address_cols):
        print("Creando la columna 'direccion_negocio'...")
        # Convertir columnas de dirección a string y rellenar NaN con vacío
        edificio_str = df['edificio'].fillna('').astype(str)
        calle_str = df['calle'].fillna('').astype(str)
        distrito_str = df['distrito'].fillna('').astype(str)
        # Convertir codigo_postal a string, manejando posibles no números y NaN
        codigo_postal_str = df['codigo_postal'].apply(lambda x: str(int(x)) if pd.notna(x) and isinstance(x, (int, float)) else str(x) if pd.notna(x) else '').astype(str)

        # Crear la columna combinada, asegurando espacios correctos y quitando comas innecesarias
        df['direccion_negocio'] = (edificio_str + ' ' + calle_str + ', ' + distrito_str + ' ' + codigo_postal_str).str.replace(r'\s+,', ',', regex=True).str.replace(r',\s*$', '', regex=True).str.replace(r'^\s*,\s*', '', regex=True).str.replace(r'^\s+', '', regex=True).str.strip()
        # Reemplazar direcciones que solo contienen comas o espacios vacíos con NaN
        df['direccion_negocio'] = df['direccion_negocio'].replace(r'^[,\s]*$', np.nan, regex=True)
        print("Columna 'direccion_negocio' creada.")

    else:
        print("Advertencia: No se encontraron todas las columnas necesarias para crear 'direccion_negocio' ('edificio', 'calle', 'distrito', 'codigo_postal'). Se creará como NaN.")
        missing_cols = [col for col in address_cols if col not in df.columns]
        print(f"Columnas faltantes: {missing_cols}")
        df['direccion_negocio'] = np.nan

    # Añadir la nueva columna a la lista de columnas a mantener si se creó
    if 'direccion_negocio' in df.columns and 'direccion_negocio' not in columns_to_keep:
         columns_to_keep.append('direccion_negocio')

    # Verificar si todas las columnas a mantener existen en el DataFrame
    existing_columns_to_keep = [col for col in columns_to_keep if col in df.columns]
    missing_final_cols = [col for col in columns_to_keep if col not in df.columns]
    if missing_final_cols:
        print(f"Advertencia: Las siguientes columnas solicitadas no se encontraron y no estarán en el archivo final: {missing_final_cols}")

    # Seleccionar solo las columnas deseadas (y existentes)
    df_final = df[existing_columns_to_keep].copy() # Usar .copy() para evitar SettingWithCopyWarning
    print(f"Columnas seleccionadas para el archivo final: {existing_columns_to_keep}")

    # Guardar el DataFrame procesado en un nuevo archivo CSV
    df_final.to_csv(output_file_path, index=False, encoding='utf-8') # Especificar encoding utf-8
    print(f"Archivo procesado guardado exitosamente en: {output_file_path}")

except FileNotFoundError:
    print(f"Error: El archivo no fue encontrado en {input_file_path}")
except pd.errors.EmptyDataError:
    print(f"Error: El archivo en {input_file_path} está vacío.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Leyendo archivo: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_restaurantes_con_ids.csv
Archivo leído correctamente.
Creando la columna 'direccion_negocio'...
Columna 'direccion_negocio' creada.
Columnas seleccionadas para el archivo final: ['id_nombre', 'nombre_restaurante', 'calificacion', 'puntuacion', 'tipo_cocina', 'distrito', 'edificio', 'calle', 'codigo_postal', 'telefono', 'latitud', 'longitud', 'punto_ubicacion', 'distrito_comunitario', 'distrito_consejo', 'zona_censo', 'fecha_inspeccion', 'direccion_negocio']
Archivo procesado guardado exitosamente en: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_restaurantes_con_ids_procesado.csv
